# Preparations

### Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from theano import scan
import theano.tensor as T
import pymc3 as pm
import theano
from sklearn.metrics import r2_score
import seaborn as sns
import os, sys, subprocess

### Load Real Data

In [ ]:
data = pd.read_csv(os.path.join('../data/', "2nd NOS Modeling Training Data OS1.csv"))

In [ ]:
data['DV'] = ((data['DV'].values - 1) / 2) - 1

observed_R = data.pivot(columns = 'ID', index = 'trialseq', values = 'DV').values[:, np.newaxis, :]

### Occasion Setting Model

##### In its default form, the code block below allows λbar, γ1, and γ2 to scale per their formulas. However, in order to improve α parameter estimation (as described in our report), please set each of these to 1. Please see comments in code below on which code to activate and deactivate by highlighting that code and pressing ctrl / (or, on Mac, command /). 

In [ ]:
def learning_function(stimuli_shown, Λ, λ, prev_V, prev_Vbar, prev_P, prev_N, stimulus_type, OS1_type, αA, αAB, αABC, αR, αB, αC, αBC, αG, αH, αJ, αK, αJK, αT, αTJ, αTJK):
    
    Λbar = T.zeros_like(Λ)
    Λbar = T.inc_subtensor(Λbar[0,:], (prev_V[4,:] > 0) * (1 - Λ[0, :])) #Acs
    Λbar = T.inc_subtensor(Λbar[1,:], (prev_V[2,:] > 0) * (1 - Λ[2, :])) #A1
    Λbar = T.inc_subtensor(Λbar[2,:], (prev_V[2,:] > 0) * (1 - Λ[4, :])) #Bcs
    Λbar = T.inc_subtensor(Λbar[3,:], (prev_V[4,:] > 0) * (1 - Λ[4, :])) #B1
    Λbar = T.inc_subtensor(Λbar[4,:], (prev_V[4,:] > 0) * (1 - Λ[4, :])) #C
    Λbar = T.inc_subtensor(Λbar[5,:], (prev_V[5,:] > 0) * (1 - Λ[5, :])) #R
    Λbar = T.inc_subtensor(Λbar[6,:], (prev_V[6,:] > 0) * (1 - Λ[6, :])) #G
    Λbar = T.inc_subtensor(Λbar[7,:], (prev_V[6,:] > 0) * (1 - Λ[6, :])) #H
    Λbar = T.inc_subtensor(Λbar[8,:], (prev_V[8,:] > 0) * (1 - Λ[10, :])) #Jcs
    Λbar = T.inc_subtensor(Λbar[9,:], (prev_V[10,:] > 0) * (1 - Λ[10, :])) #J1
    Λbar = T.inc_subtensor(Λbar[10,:], (prev_V[10,:] > 0) * (1 - Λ[10, :])) #K
    Λbar = T.inc_subtensor(Λbar[11,:], (prev_V[11,:] > 0) * (1 - Λ[11, :])) #Tcs
    Λbar = T.inc_subtensor(Λbar[12,:], (prev_V[8,:] > 0) * (1 - Λ[9, :])) #T1
    Λbar = T.inc_subtensor(Λbar[13,:], (prev_V[2,:] > 0) * (1 - Λ[2, :])) #A1_abs
    Λbar = T.inc_subtensor(Λbar[14,:], (prev_V[4,:] > 0) * (1 - Λ[4, :])) #B1_abs
    Λbar = T.inc_subtensor(Λbar[15,:], (prev_V[10,:] > 0) * (1 - Λ[10, :])) #J1_abs
    Λbar = T.inc_subtensor(Λbar[16,:], (prev_V[8,:] > 0) * (1 - Λ[9, :])) #T1_abs
    
    #To make λbar = 1, activate "λbar = T.ones_like(Λbar)" code below and 
    #deactivate all "λbar =" code under "Code for λbar Scaling" below.
    
    #Code for λbar = 1
#     λbar = T.ones_like(Λbar)
    
    #Code for λbar Scaling
    λbar = T.zeros_like(Λbar)
    λbar = T.inc_subtensor(λbar[0,:], prev_V[0,:]) #Acs
    λbar = T.inc_subtensor(λbar[1,:], prev_V[2,:]) #A1
    λbar = T.inc_subtensor(λbar[2,:], prev_V[4,:]) #Bcs
    λbar = T.inc_subtensor(λbar[3,:], prev_V[4,:]) #B1
    λbar = T.inc_subtensor(λbar[4,:], prev_V[4,:]) #C
    λbar = T.inc_subtensor(λbar[5,:], prev_V[5,:]) #R
    λbar = T.inc_subtensor(λbar[6,:], prev_V[6,:]) #G
    λbar = T.inc_subtensor(λbar[7,:], prev_V[6,:]) #H
    λbar = T.inc_subtensor(λbar[8,:], prev_V[10,:]) #Jcs
    λbar = T.inc_subtensor(λbar[9,:], prev_V[10,:]) #J1
    λbar = T.inc_subtensor(λbar[10,:], prev_V[10,:]) #K
    λbar = T.inc_subtensor(λbar[11,:], prev_V[11,:]) #Tcs
    λbar = T.inc_subtensor(λbar[12,:], prev_V[9,:]) #T1
    λbar = T.inc_subtensor(λbar[13,:], prev_V[2,:]) #A1_abs
    λbar = T.inc_subtensor(λbar[14,:], prev_V[4,:]) #B1_abs
    λbar = T.inc_subtensor(λbar[15,:], prev_V[10,:]) #J1_abs
    λbar = T.inc_subtensor(λbar[16,:], prev_V[8,:]) #T1_abs
    
    
    #Prediction error
    pe_V = λ - prev_V
    pe_Vbar = λbar - prev_Vbar
    pe_P = λ - prev_P
    pe_N = λbar - prev_N
    
    #Stimulus-Specific α
    αA = αA*(stimuli_shown[17,:] > 0)
    αAB = αAB*(stimuli_shown[18,:] > 0)
    αABC = αABC*(stimuli_shown[19,:] > 0)
    αR = αR*(stimuli_shown[20,:] > 0)
    αB = αB*(stimuli_shown[21,:] > 0)
    αC = αC*(stimuli_shown[22,:] > 0)
    αBC = αBC*(stimuli_shown[23,:] > 0)
    αG = αG*(stimuli_shown[24,:] > 0)
    αH = αH*(stimuli_shown[25,:] > 0)
    αJ = αJ*(stimuli_shown[26,:] > 0)
    αK = αK*(stimuli_shown[27,:] > 0)
    αJK = αJK*(stimuli_shown[28,:] > 0)
    αT = αT*(stimuli_shown[29,:] > 0)
    αTJ = αTJ*(stimuli_shown[30,:] > 0)
    αTJK = αTJK*(stimuli_shown[31,:] > 0)

    #To make γ = 1, activate code under "Code for γ = 1" and 
    #deactivate code under "Code for γ Scaling" below.
    
    #Code for γ = 1
#     prev_γ1 = T.zeros_like(prev_V)
#     prev_γ1 = T.set_subtensor(prev_γ1[stimulus_type.nonzero(),:],1)
    
    #Code for γ Scaling
    prev_γ1 = prev_V * prev_Vbar

    #Delta formulas
    CS_prev_γ1 = (prev_γ1 * stimuli_shown).sum(axis=0)
    ΔV = Λ * (αA + αAB + αABC + αR + αB + αC + αBC + αG + αH + αJ + αK + αJK + αT + αTJ + αTJK) * pe_V
    ΔVbar = Λbar * (αA + αAB + αABC + αR + αB + αC + αBC + αG + αH + αJ + αK + αJK + αT + αTJ + αTJK) * pe_Vbar
    ΔP = Λ * (αA + αAB + αABC + αR + αB + αC + αBC + αG + αH + αJ + αK + αJK + αT + αTJ + αTJK) * CS_prev_γ1 * pe_P
    ΔN = Λbar * (αA + αAB + αABC + αR + αB + αC + αBC + αG + αH + αJ + αK + αJK + αT + αTJ + αTJK) * CS_prev_γ1 * pe_N


    # Only update stimuli that were shown
    ΔV = ΔV * stimuli_shown
    ΔVbar = ΔVbar * stimuli_shown
    ΔP = ΔP * stimuli_shown
    ΔN = ΔN * stimuli_shown
    
    # Update V, Vbar, P, N, P2, N2
    V = T.zeros_like(prev_V)
    Vbar = T.zeros_like(prev_Vbar)
    P = T.zeros_like(prev_P)
    N = T.zeros_like(prev_N)
    
    # Only update V and Vbar for CSs. Only update P and N for 1st-order OSs. Only update P2 and N2 for 2nd-order OSs.
    V = T.inc_subtensor(V[T.eq(stimulus_type, 1)], prev_V[T.eq(stimulus_type, 1)] + ΔV[T.eq(stimulus_type, 1)])
    Vbar = T.inc_subtensor(Vbar[T.eq(stimulus_type, 1)], prev_Vbar[T.eq(stimulus_type, 1)] + ΔVbar[T.eq(stimulus_type, 1)])
    P = T.inc_subtensor(P[T.eq(OS1_type, 1)], prev_P[T.eq(OS1_type, 1)] + ΔP[T.eq(OS1_type, 1)])
    N = T.inc_subtensor(N[T.eq(OS1_type, 1)], prev_N[T.eq(OS1_type, 1)] + ΔN[T.eq(OS1_type, 1)])
    
    return V, Vbar, P, N

### Generate Simulated Data with Model

##### In code below, use "Code for randomized α values" to simulate random α values, which will then be estimated by our model. This was used for our simulated vs recovered plots in the manuscript. In order to graph "perfect" learning rate data, use "Code for α = 1," which was plotted in our manuscript alongside participants with low, medium, and high α parameters.

In [ ]:
n_stim = 32
n_subjects = len(data['ID'].unique())

#Initial values
R = np.zeros((n_stim, n_subjects))
overall_R = np.zeros((1, n_subjects))
v_excitatory = np.zeros((n_stim, n_subjects))
v_inhibitory = np.zeros((n_stim, n_subjects))
P = np.zeros((n_stim, n_subjects))
N = np.zeros((n_stim, n_subjects))


#Code for randomized α values
gen_dist = pm.Beta.dist(2, 2, shape = n_subjects)
αA_subject_sim = gen_dist.random()
αAB_subject_sim = gen_dist.random()
αABC_subject_sim = gen_dist.random()
αR_subject_sim = gen_dist.random()
αB_subject_sim = gen_dist.random()
αC_subject_sim = gen_dist.random()
αBC_subject_sim = gen_dist.random()
αG_subject_sim = gen_dist.random()
αH_subject_sim = gen_dist.random()
αJ_subject_sim = gen_dist.random()
αK_subject_sim = gen_dist.random()
αJK_subject_sim = gen_dist.random()
αT_subject_sim = gen_dist.random()
αTJ_subject_sim = gen_dist.random()
αTJK_subject_sim = gen_dist.random()


#Code for α = 1
# gen_dist = pm.Beta.dist(2, 2, shape=n_subjects)
# α_subject_sim = np.ones(n_subjects)
# αA_subject_sim = np.ones(n_subjects)
# αAB_subject_sim = np.ones(n_subjects)
# αABC_subject_sim = np.ones(n_subjects)
# αR_subject_sim = np.ones(n_subjects)
# αB_subject_sim = np.ones(n_subjects)
# αC_subject_sim = np.ones(n_subjects)
# αBC_subject_sim = np.ones(n_subjects)
# αG_subject_sim = np.ones(n_subjects)
# αH_subject_sim = np.ones(n_subjects)
# αJ_subject_sim = np.ones(n_subjects)
# αK_subject_sim = np.ones(n_subjects)
# αJK_subject_sim = np.ones(n_subjects)
# αT_subject_sim = np.ones(n_subjects)
# αTJ_subject_sim = np.ones(n_subjects)
# αTJK_subject_sim = np.ones(n_subjects)

#US values
small_lambda = data.pivot(index='trialseq', values='US', columns='ID').values[:, np.newaxis, :].repeat(n_stim, axis=1).astype(float)
stim_data = []

for sub in data['ID'].unique():
    stim_data.append(data.loc[data['ID'] == sub, ['Acs', 'A1', 'Bcs', 'B1', 'C', 'R', 'G', 'H', 'Jcs', 
                                                  'J1', 'K', 'Tcs', 'T1', 'A1_abs', 'B1_abs', 
                                                  'J1_abs', 'T1_abs',
                                                  'alpha_A', 'alpha_AB', 'alpha_ABC', 'alpha_R', 'alpha_B', 'alpha_C',
                                                  'alpha_BC', 'alpha_G', 'alpha_H', 'alpha_J', 'alpha_K', 'alpha_JK',
                                                  'alpha_T', 'alpha_TJ', 'alpha_TJK']].values)

stimuli_shown = np.dstack(stim_data)
big_lambda = small_lambda

#Add imaginary -1th trial
big_lambda = np.vstack([np.zeros((1, n_stim, n_subjects)), big_lambda[:-1, ...]]).astype(float) # Add one trial of zeros to the start, remove the last trial
small_lambda = big_lambda
stimuli_shown = np.vstack([np.zeros((1, n_stim, n_subjects)), stimuli_shown]) # Add one trial of zeros to the start, DO NOT remove the last trial - this is needed for prediction

#Designate stimulus types
stimulus_type = np.ones(n_stim)
stimulus_type[[1, 3, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]] = 0 #make all OSs = 0

OS1_type = np.zeros(n_stim)
OS1_type[[1, 3, 9, 12, 13, 14, 15, 16]] = 1 #make 1st OSs = 1


#Convert task outcomes to tensors
big_lambda = T.as_tensor_variable(big_lambda.astype(float))
small_lambda = T.as_tensor_variable(small_lambda.astype(float))
stimuli_shown = T.as_tensor_variable(stimuli_shown)

stimuli_shown_sim = stimuli_shown.copy()
big_lambda_sim = big_lambda.copy()
small_lambda_sim = small_lambda.copy()

### Run Fake Data Simulation

In [ ]:
#Run the loop
output, updates = scan(fn=learning_function,
                    sequences=[{'input': stimuli_shown_sim[:-1, ...]},
                             {'input': big_lambda_sim},
                             {'input': small_lambda_sim}],
                    outputs_info=[v_excitatory, v_inhibitory, P, N],
                    non_sequences = [stimulus_type, OS1_type, αA_subject_sim, αAB_subject_sim, αABC_subject_sim, αR_subject_sim, αB_subject_sim, αC_subject_sim, αBC_subject_sim, αG_subject_sim, αH_subject_sim, αJ_subject_sim, αK_subject_sim, αJK_subject_sim, αT_subject_sim, αTJ_subject_sim, αTJK_subject_sim])

#Get model output
V_out, Vbar_out, P_out, N_out = [i.eval() for i in output]

estimated_overall_R = ((V_out * stimuli_shown_sim[1:, ...]).sum(axis=1) - (Vbar_out * stimuli_shown_sim[1:, ...]).sum(axis=1)) + \
    ((P_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (Vbar_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (V_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (Vbar_out * stimuli_shown_sim[1:, ...]).sum(axis=1)) - \
    ((N_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (V_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (V_out * stimuli_shown_sim[1:, ...]).sum(axis=1) * (Vbar_out * stimuli_shown_sim[1:, ...]).sum(axis=1))

overall_R_sim = estimated_overall_R.eval()

### Check parameter recovery

In [ ]:
n_subjects = len(data['ID'].unique())

#Initial values
R = np.zeros((n_stim, n_subjects))

#US values
small_lambda = data.pivot(index='trialseq', values='US', columns='ID').values[:, np.newaxis, :].repeat(n_stim, axis=1).astype(float)
stim_data = []

for sub in data['ID'].unique():
    stim_data.append(data.loc[data['ID'] == sub, ['Acs', 'A1', 'Bcs', 'B1', 'C', 'R', 'G', 'H', 'Jcs', 
                                                  'J1', 'K', 'Tcs', 'T1', 'A1_abs', 'B1_abs', 
                                                  'J1_abs', 'T1_abs',
                                                  'alpha_A', 'alpha_AB', 'alpha_ABC', 'alpha_R', 'alpha_B', 'alpha_C',
                                                  'alpha_BC', 'alpha_G', 'alpha_H', 'alpha_J', 'alpha_K', 'alpha_JK',
                                                  'alpha_T', 'alpha_TJ', 'alpha_TJK']].values)

stimuli_shown = np.dstack(stim_data)
big_lambda = small_lambda

#Add imaginary -1th trial
big_lambda = np.vstack([np.zeros((1, n_stim, n_subjects)), big_lambda[:-1, ...]]).astype(float) # Add one trial of zeros to the start, remove the last trial
small_lambda = big_lambda
stimuli_shown = np.vstack([np.zeros((1, n_stim, n_subjects)), stimuli_shown]) # Add one trial of zeros to the start, DO NOT remove the last trial - this is needed for prediction

stimulus_type = np.ones(n_stim)
stimulus_type[[1, 3, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]] = 0 #make all OSs = 0

OS1_type = np.zeros(n_stim)
OS1_type[[1, 3, 9, 12, 13, 14, 15, 16]] = 1 #make 1st OSs = 1

#Convert task outcomes to tensors
big_lambda = T.as_tensor_variable(big_lambda.astype(float))
small_lambda = T.as_tensor_variable(small_lambda.astype(float))
stimuli_shown = T.as_tensor_variable(stimuli_shown)

with pm.Model() as model:
    
    # Learning rate lies between 0 and 1, so we use a beta distribution
    αA_mean = pm.Normal('αA_mean', 0.5, 10)
    αA_sd = pm.HalfCauchy('αA_sd', 10)
    
    αAB_mean = pm.Normal('αAB_mean', 0.5, 10)
    αAB_sd = pm.HalfCauchy('αAB_sd', 10)
    
    αABC_mean = pm.Normal('αABC_mean', 0.5, 10)
    αABC_sd = pm.HalfCauchy('αABC_sd', 10)
    
    αR_mean = pm.Normal('αR_mean', 0.5, 10)
    αR_sd = pm.HalfCauchy('αR_sd', 10)
    
    αB_mean = pm.Normal('αB_mean', 0.5, 10)
    αB_sd = pm.HalfCauchy('αB_sd', 10)
    
    αC_mean = pm.Normal('αC_mean', 0.5, 10)
    αC_sd = pm.HalfCauchy('αC_sd', 10)
    
    αBC_mean = pm.Normal('αBC_mean', 0.5, 10)
    αBC_sd = pm.HalfCauchy('αBC_sd', 10)
    
    αG_mean = pm.Normal('αG_mean', 0.5, 10)
    αG_sd = pm.HalfCauchy('αG_sd', 10)
    
    αH_mean = pm.Normal('αH_mean', 0.5, 10)
    αH_sd = pm.HalfCauchy('αH_sd', 10)
    
    αJ_mean = pm.Normal('αJ_mean', 0.5, 10)
    αJ_sd = pm.HalfCauchy('αJ_sd', 10)
    
    αK_mean = pm.Normal('αK_mean', 0.5, 10)
    αK_sd = pm.HalfCauchy('αK_sd', 10)
    
    αJK_mean = pm.Normal('αJK_mean', 0.5, 10)
    αJK_sd = pm.HalfCauchy('αJK_sd', 10)
    
    αT_mean = pm.Normal('αT_mean', 0.5, 10)
    αT_sd = pm.HalfCauchy('αT_sd', 10)
    
    αTJ_mean = pm.Normal('αTJ_mean', 0.5, 10)
    αTJ_sd = pm.HalfCauchy('αTJ_sd', 10)
    
    αTJK_mean = pm.Normal('αTJK_mean', 0.5, 10)
    αTJK_sd = pm.HalfCauchy('αTJK_sd', 10)

    
    BoundedNormal = pm.Bound(pm.Normal, lower=0, upper=1)
    αA_subject = BoundedNormal('αA', mu=αA_mean, sd=αA_sd, shape=(n_subjects,))
    αAB_subject = BoundedNormal('αAB', mu=αAB_mean, sd=αAB_sd, shape=(n_subjects,))
    αABC_subject = BoundedNormal('αABC', mu=αABC_mean, sd=αABC_sd, shape=(n_subjects,))
    αR_subject = BoundedNormal('αR', mu=αR_mean, sd=αR_sd, shape=(n_subjects,))
    αB_subject = BoundedNormal('αB', mu=αB_mean, sd=αB_sd, shape=(n_subjects,))
    αC_subject = BoundedNormal('αC', mu=αC_mean, sd=αC_sd, shape=(n_subjects,))
    αBC_subject = BoundedNormal('αBC', mu=αBC_mean, sd=αBC_sd, shape=(n_subjects,))
    αG_subject = BoundedNormal('αG', mu=αG_mean, sd=αG_sd, shape=(n_subjects,))
    αH_subject = BoundedNormal('αH', mu=αH_mean, sd=αH_sd, shape=(n_subjects,))
    αJ_subject = BoundedNormal('αJ', mu=αJ_mean, sd=αJ_sd, shape=(n_subjects,))
    αK_subject = BoundedNormal('αK', mu=αK_mean, sd=αK_sd, shape=(n_subjects,))
    αJK_subject = BoundedNormal('αJK', mu=αJK_mean, sd=αJK_sd, shape=(n_subjects,))
    αT_subject = BoundedNormal('αT', mu=αT_mean, sd=αT_sd, shape=(n_subjects,))
    αTJ_subject = BoundedNormal('αTJ', mu=αTJ_mean, sd=αTJ_sd, shape=(n_subjects,))
    αTJK_subject = BoundedNormal('αTJK', mu=αTJK_mean, sd=αTJK_sd, shape=(n_subjects,))

    
# Run the loop
    output, updates = scan(fn=learning_function,
                      sequences=[dict(input=stimuli_shown[:-1, ...]), dict(input=big_lambda), dict(input=small_lambda)],
                      outputs_info=[v_excitatory, v_inhibitory, P, N],
                      non_sequences = [stimulus_type, OS1_type, αA_subject, αAB_subject, αABC_subject, αR_subject, αB_subject, αC_subject, αBC_subject, αG_subject, αH_subject, αJ_subject, αK_subject, αJK_subject, αT_subject, αTJ_subject, αTJK_subject])
    
    # Get model output
    V, Vbar, P, N = output
    
    # Calculate response - combine value learning and occasion setting
    R = (V - Vbar) + ((P * Vbar * V*Vbar) - (N * V *V*Vbar))

    # # Single R value
    estimated_overall_R = ((V * stimuli_shown[1:, ...]).sum(axis=1) - (Vbar * stimuli_shown[1:, ...]).sum(axis=1)) + \
        ((P * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1)) - \
        ((N * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1))
   
    # This allows us to output the estimated R
    estimated_overall_R = pm.Deterministic('estimated_overall_R', estimated_overall_R)
    
    # Reshape output of the model and get categorical likelihood
    sigma = pm.HalfCauchy('sigma', 0.5)
    likelihood = pm.Normal('likelihood', mu=estimated_overall_R, sigma=sigma, observed=pd.DataFrame(overall_R_sim.squeeze()))

# Simulated Data (go to "Real Data" below if you want to skip data simulations and just get real data results)

### Fit the Model

#### Variational Inference

In [ ]:
from pymc3.variational.callbacks import CheckParametersConvergence
with model:
    approx = pm.fit(method='advi', n=40000, callbacks=[CheckParametersConvergence()])
trace = approx.sample(1000)

In [ ]:
alpha_output = pm.summary(trace, kind='stats', varnames=[i for i in model.named_vars if 'α' in i and not i in model.deterministics and not 'log' in i and not 'interval' in i])

In [ ]:
pm.traceplot(trace, var_names=['αA_mean', 'αAB_mean', 'αABC_mean', 'αR_mean', 'αB_mean', 'αC_mean', 'αBC_mean', 'αG_mean', 'αH_mean', 'αJ_mean', 'αK_mean', 'αJK_mean', 'αT_mean', 'αTJ_mean', 'αTJK_mean']);

In [ ]:
recovered_data_var = {'Simulated_αA': αA_subject_sim, 'Recovered_αA': trace['αA'].mean(axis=0), 
                      'Simulated_αAB': αAB_subject_sim, 'Recovered_αAB': trace['αAB'].mean(axis=0),
                      'Simulated_αABC': αABC_subject_sim, 'Recovered_αABC': trace['αABC'].mean(axis=0),
                      'Simulated_αR': αR_subject_sim, 'Recovered_αR': trace['αR'].mean(axis=0),
                      'Simulated_αB': αB_subject_sim, 'Recovered_αB': trace['αB'].mean(axis=0),
                      'Simulated_αC': αC_subject_sim, 'Recovered_αC': trace['αC'].mean(axis=0),
                      'Simulated_αBC': αBC_subject_sim, 'Recovered_αBC': trace['αBC'].mean(axis=0),
                      'Simulated_αG': αG_subject_sim, 'Recovered_αG': trace['αG'].mean(axis=0),
                      'Simulated_αH': αH_subject_sim, 'Recovered_αH': trace['αH'].mean(axis=0),
                      'Simulated_αJ': αJ_subject_sim, 'Recovered_αJ': trace['αJ'].mean(axis=0),
                      'Simulated_αK': αK_subject_sim, 'Recovered_αK': trace['αK'].mean(axis=0),
                      'Simulated_αJK': αJK_subject_sim, 'Recovered_αJK': trace['αJK'].mean(axis=0),
                      'Simulated_αT': αT_subject_sim, 'Recovered_αT': trace['αT'].mean(axis=0),
                      'Simulated_αTJ': αTJ_subject_sim, 'Recovered_αTJ': trace['αTJ'].mean(axis=0),
                      'Simulated_αTJK': αTJK_subject_sim, 'Recovered_αTJK': trace['αTJK'].mean(axis=0)}

recovered_data_var = pd.DataFrame(recovered_data_var)
recovered_data_var.to_csv(os.path.join('../output/',r'2nd NOS - OS1 Stimulus-Specific, Simulated vs Recovered.csv'))

In [ ]:
f, ax = plt.subplots(3, 5, sharex = True, sharey = True, figsize=(12, 7.5))
f.suptitle('Simulated vs Recovered α Parameters', y=1.02, fontsize = 16)
f.text(.5, -.02, 'Simulated α', va='center', ha='center', fontsize = 16)
f.text(-.02, .5, 'Recovered α', va='center', ha='center', fontsize = 16, rotation=90)

sns.regplot(αA_subject_sim, trace['αA'].mean(axis=0), label='αA_subject', ax=ax[0,0], color = 'black')
sns.regplot(αAB_subject_sim, trace['αAB'].mean(axis=0), label='αAB_subject', ax=ax[0,1], color = 'black')
sns.regplot(αABC_subject_sim, trace['αABC'].mean(axis=0), label='αABC_subject', ax=ax[0,2], color = 'black')
sns.regplot(αR_subject_sim, trace['αR'].mean(axis=0), label='αR_subject', ax=ax[0,3], color = 'black')
sns.regplot(αB_subject_sim, trace['αB'].mean(axis=0), label='αB_subject', ax=ax[0,4], color = 'black')
sns.regplot(αC_subject_sim, trace['αC'].mean(axis=0), label='αC_subject', ax=ax[1,0], color = 'black')
sns.regplot(αBC_subject_sim, trace['αBC'].mean(axis=0), label='αBC_subject', ax=ax[1,1], color = 'black')
sns.regplot(αG_subject_sim, trace['αG'].mean(axis=0), label='αG_subject', ax=ax[1,2], color = 'black')
sns.regplot(αH_subject_sim, trace['αH'].mean(axis=0), label='αH_subject', ax=ax[1,3], color = 'black')
sns.regplot(αJ_subject_sim, trace['αJ'].mean(axis=0), label='αJ_subject', ax=ax[1,4], color = 'black')
sns.regplot(αK_subject_sim, trace['αK'].mean(axis=0), label='αK_subject', ax=ax[2,0], color = 'black')
sns.regplot(αJK_subject_sim, trace['αJK'].mean(axis=0), label='αJK_subject', ax=ax[2,1], color = 'black')
sns.regplot(αT_subject_sim, trace['αT'].mean(axis=0), label='αT_subject', ax=ax[2,2], color = 'black')
sns.regplot(αTJ_subject_sim, trace['αTJ'].mean(axis=0), label='αTJ_subject', ax=ax[2,3], color = 'black')
sns.regplot(αTJK_subject_sim, trace['αTJK'].mean(axis=0), label='αTJK_subject', ax=ax[2,4], color = 'black')

for i in range(3):
    for j in range(5):
        ax[0,0].set_title('α A')
        ax[0,1].set_title('α AB')
        ax[0,2].set_title('α ABC')
        ax[0,3].set_title('α R')
        ax[0,4].set_title('α B')
        ax[1,0].set_title('α C')
        ax[1,1].set_title('α BC')
        ax[1,2].set_title('α G')
        ax[1,3].set_title('α H')
        ax[1,4].set_title('α J')
        ax[2,0].set_title('α K')
        ax[2,1].set_title('α JK')
        ax[2,2].set_title('α T')
        ax[2,3].set_title('α TJ')
        ax[2,4].set_title('α TJK')

plt.setp(ax, xticks=[0, .2, .4, .6, .8, 1], yticks=[0, .2, .4, .6, .8, 1])        
plt.tight_layout()

plt.savefig(os.path.join('../output/',r'2nd NOS - OS1 Stimulus-Specific, Simulated vs Recovered.svg'), bbox_inches='tight')

# Real Data

### Fit the Model to Real Data

In [ ]:
n_subjects = len(data['ID'].unique())

# Initial values
R = np.zeros((n_stim, n_subjects))  # Value estimate
overall_R = np.zeros((1, n_subjects))
v_excitatory = np.zeros((n_stim, n_subjects)) 
v_inhibitory = np.zeros((n_stim, n_subjects)) 
P = np.zeros((n_stim, n_subjects))
N = np.zeros((n_stim, n_subjects))
P2 = np.zeros((n_stim, n_subjects))
N2 = np.zeros((n_stim, n_subjects))
gamma = np.ones((n_stim, n_subjects))

# US values
small_lambda = data.pivot(index='trialseq', values='US', columns='ID').values[:, np.newaxis, :].repeat(n_stim, axis=1)
stim_data = []

for sub in data['ID'].unique():
    stim_data.append(data.loc[data['ID'] == sub, ['Acs', 'A1', 'Bcs', 'B1', 'C', 'R', 'G', 'H', 'Jcs', 
                                                  'J1', 'K', 'Tcs', 'T1', 'A1_abs', 'B1_abs', 
                                                  'J1_abs', 'T1_abs',
                                                  'alpha_A', 'alpha_AB', 'alpha_ABC', 'alpha_R', 'alpha_B', 'alpha_C',
                                                  'alpha_BC', 'alpha_G', 'alpha_H', 'alpha_J', 'alpha_K', 'alpha_JK',
                                                  'alpha_T', 'alpha_TJ', 'alpha_TJK']].values)
    
stimuli_shown = np.dstack(stim_data)
big_lambda = small_lambda

# Add imaginary -1th trial
big_lambda = np.vstack([np.zeros((1, n_stim, n_subjects)), big_lambda[:-1, ...]])  # Add one trial of zeros to the start, remove the last trial
small_lambda = big_lambda
stimuli_shown = np.vstack([np.zeros((1, n_stim, n_subjects)), stimuli_shown]) # Add one trial of zeros to the start, DO NOT remove the last trial - this is needed for prediction

stimulus_type = np.ones(n_stim)
stimulus_type[[1, 3, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]] = 0 #make all OSs = 0

OS1_type = np.zeros(n_stim)
OS1_type[[1, 3, 9, 12, 13, 14, 15, 16]] = 1 #make 1st OSs = 1

# Convert task outcomes to tensors
big_lambda = T.as_tensor_variable(big_lambda)
small_lambda = T.as_tensor_variable(small_lambda)
stimuli_shown = T.as_tensor_variable(stimuli_shown)

with pm.Model() as model:
    
    # Learning rate lies between 0 and 1 so we use a beta distribution
    αA_mean = pm.Normal('αA_mean', 0.5, 10)
    αA_sd = pm.HalfCauchy('αA_sd', 10)
    
    αAB_mean = pm.Normal('αAB_mean', 0.5, 10)
    αAB_sd = pm.HalfCauchy('αAB_sd', 10)
    
    αABC_mean = pm.Normal('αABC_mean', 0.5, 10)
    αABC_sd = pm.HalfCauchy('αABC_sd', 10)
    
    αR_mean = pm.Normal('αR_mean', 0.5, 10)
    αR_sd = pm.HalfCauchy('αR_sd', 10)
    
    αB_mean = pm.Normal('αB_mean', 0.5, 10)
    αB_sd = pm.HalfCauchy('αB_sd', 10)
    
    αC_mean = pm.Normal('αC_mean', 0.5, 10)
    αC_sd = pm.HalfCauchy('αC_sd', 10)
    
    αBC_mean = pm.Normal('αBC_mean', 0.5, 10)
    αBC_sd = pm.HalfCauchy('αBC_sd', 10)
    
    αG_mean = pm.Normal('αG_mean', 0.5, 10)
    αG_sd = pm.HalfCauchy('αG_sd', 10)
    
    αH_mean = pm.Normal('αH_mean', 0.5, 10)
    αH_sd = pm.HalfCauchy('αH_sd', 10)
    
    αJ_mean = pm.Normal('αJ_mean', 0.5, 10)
    αJ_sd = pm.HalfCauchy('αJ_sd', 10)
    
    αK_mean = pm.Normal('αK_mean', 0.5, 10)
    αK_sd = pm.HalfCauchy('αK_sd', 10)
    
    αJK_mean = pm.Normal('αJK_mean', 0.5, 10)
    αJK_sd = pm.HalfCauchy('αJK_sd', 10)
    
    αT_mean = pm.Normal('αT_mean', 0.5, 10)
    αT_sd = pm.HalfCauchy('αT_sd', 10)
    
    αTJ_mean = pm.Normal('αTJ_mean', 0.5, 10)
    αTJ_sd = pm.HalfCauchy('αTJ_sd', 10)
    
    αTJK_mean = pm.Normal('αTJK_mean', 0.5, 10)
    αTJK_sd = pm.HalfCauchy('αTJK_sd', 10)

    
    BoundedNormal = pm.Bound(pm.Normal, lower=0, upper=1)
    αA_subject = BoundedNormal('αA', mu=αA_mean, sd=αA_sd, shape=(n_subjects,))
    αAB_subject = BoundedNormal('αAB', mu=αAB_mean, sd=αAB_sd, shape=(n_subjects,))
    αABC_subject = BoundedNormal('αABC', mu=αABC_mean, sd=αABC_sd, shape=(n_subjects,))
    αR_subject = BoundedNormal('αR', mu=αR_mean, sd=αR_sd, shape=(n_subjects,))
    αB_subject = BoundedNormal('αB', mu=αB_mean, sd=αB_sd, shape=(n_subjects,))
    αC_subject = BoundedNormal('αC', mu=αC_mean, sd=αC_sd, shape=(n_subjects,))
    αBC_subject = BoundedNormal('αBC', mu=αBC_mean, sd=αBC_sd, shape=(n_subjects,))
    αG_subject = BoundedNormal('αG', mu=αG_mean, sd=αG_sd, shape=(n_subjects,))
    αH_subject = BoundedNormal('αH', mu=αH_mean, sd=αH_sd, shape=(n_subjects,))
    αJ_subject = BoundedNormal('αJ', mu=αJ_mean, sd=αJ_sd, shape=(n_subjects,))
    αK_subject = BoundedNormal('αK', mu=αK_mean, sd=αK_sd, shape=(n_subjects,))
    αJK_subject = BoundedNormal('αJK', mu=αJK_mean, sd=αJK_sd, shape=(n_subjects,))
    αT_subject = BoundedNormal('αT', mu=αT_mean, sd=αT_sd, shape=(n_subjects,))
    αTJ_subject = BoundedNormal('αTJ', mu=αTJ_mean, sd=αTJ_sd, shape=(n_subjects,))
    αTJK_subject = BoundedNormal('αTJK', mu=αTJK_mean, sd=αTJK_sd, shape=(n_subjects,))
    


    
    # Run the loop
    output, updates = scan(fn=learning_function,
                      sequences=[dict(input=stimuli_shown[:-1, ...]), dict(input=big_lambda), dict(input=small_lambda)],
                      outputs_info=[v_excitatory, v_inhibitory, P, N],
                      non_sequences = [stimulus_type, OS1_type, αA_subject, αAB_subject, αABC_subject, αR_subject, αB_subject, αC_subject, αBC_subject, αG_subject, αH_subject, αJ_subject, αK_subject, αJK_subject, αT_subject, αTJ_subject, αTJK_subject])
    
    # Get model output
    V, Vbar, P, N = output
    
    # Calculate response - combine value learning and occasion setting
    R = (V - Vbar) + ((P * Vbar * V*Vbar) - (N * V *V*Vbar))

    # # Single R value
    estimated_overall_R = ((V * stimuli_shown[1:, ...]).sum(axis=1) - (Vbar * stimuli_shown[1:, ...]).sum(axis=1)) + \
        ((P * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1)) - \
        ((N * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (V * stimuli_shown[1:, ...]).sum(axis=1) * (Vbar * stimuli_shown[1:, ...]).sum(axis=1))
    
    # This allows us to output the estimated R
    estimated_overall_R = pm.Deterministic('estimated_overall_R', estimated_overall_R)
    V = pm.Deterministic('estimated_V', V)
    Vbar = pm.Deterministic('estimated_Vbar', Vbar)
    P = pm.Deterministic('estimated_P', P)
    N = pm.Deterministic('estimated_N', N)
    γ1 = pm.Deterministic('estimated_γ1', V*Vbar)
          
    # Reshape output of the model and get categorical likelihood
    sigma = pm.HalfCauchy('sigma', 0.5)
    likelihood = pm.Normal('likelihood', mu=estimated_overall_R, sigma=sigma, observed=pd.DataFrame(observed_R.squeeze()))

#### Variational Inference

In [ ]:
from pymc3.variational.callbacks import CheckParametersConvergence
with model:
    approx = pm.fit(method='advi', n=40000, callbacks=[CheckParametersConvergence()])
trace = approx.sample(1000)

In [ ]:
alpha_output = pm.summary(trace, kind='stats', varnames=[i for i in model.named_vars if 'α' in i and not i in model.deterministics and not 'log' in i and not 'interval' in i])

In [ ]:
pm.traceplot(trace, var_names=['αA_mean', 'αAB_mean', 'αABC_mean', 'αR_mean', 'αB_mean', 'αC_mean', 'αBC_mean', 'αG_mean', 'αH_mean', 'αJ_mean', 'αK_mean', 'αJK_mean', 'αT_mean', 'αTJ_mean', 'αTJK_mean']);

### WAIC, R2, and Output

In [ ]:
overall_R_mean = trace['estimated_overall_R'].mean(axis=0)
overall_R_sd = trace['estimated_overall_R'].std(axis=0)

sub_ids = data['ID'].unique()

subs = [np.where(data['ID'].unique() == sub)[0][0] for sub in sub_ids]

In [ ]:
r2s = []

for n, sub in enumerate(data['ID'].unique()):
    r2s.append(r2_score(observed_R.squeeze()[~np.isnan(observed_R.squeeze()[:, n]), n], overall_R_mean[~np.isnan(observed_R.squeeze()[:, n]), n]))

group_r2 = np.median(r2s)

print(group_r2)

In [ ]:
r2s = {'R2': [r2s]}

r2s = pd.DataFrame(r2s)

r2s.to_csv(os.path.join('../output/',r'2nd NOS - OS1 Stimulus-Specific, R2 Output.csv'))

In [ ]:
waic_output = pm.waic(trace)

In [ ]:
waic_output

In [ ]:
alpha_output.to_csv(os.path.join('../output/',r'2nd NOS - OS1 Stimulus-Specific, Alpha Output.csv'))
waic_output.to_csv(os.path.join('../output/',r'2nd NOS - OS1 Stimulus-Specific, WAIC Output.csv'))

### Plotting Real, Model-Predicted, and Perfect Learning Data

In [ ]:
data = pd.read_csv(os.path.join('../data/', "2nd NOS Modeling All Data OS1.csv"))

In [ ]:
data['DV'] = ((data['DV'].values - 1) / 2) - 1

observed_R_test = data.pivot(columns = 'ID', index = 'trialseq', values = 'DV').values

##### For the following cell, you will need to have run the "Simulation Data" code near the top of this notebook using "Code for α = 1."

In [ ]:
f, ax = plt.subplots(20, 3, figsize=(36, 48), dpi = 100)

overall_R_mean = trace['estimated_overall_R'].mean(axis=0)
overall_R_sd = trace['estimated_overall_R'].std(axis=0)

sub_ids = data['ID'].unique()

subs = [np.where(data['ID'].unique() == sub)[0][0] for sub in sub_ids]
    
for n, sub in enumerate(subs):
    ax[n % 20, int(n / 20)].fill_between(range(overall_R_mean.shape[0]), overall_R_mean[:, sub] - overall_R_sd[:, sub], overall_R_mean[:, sub] + overall_R_sd[:, sub], alpha=0.3)
    ax[n % 20, int(n / 20)].plot(overall_R_mean[:, sub])
    ax[n % 20, int(n / 20)].plot(observed_R_test.squeeze()[:, sub], color='orange', linestyle='-')#participant's real data
    ax[n % 20, int(n / 20)].plot(overall_R_sim.squeeze()[:, sub], color='grey', linestyle=':', alpha = .7)#Alpha = 1; this is the correct answer if a person learned perfectly
    if n == 0:
        ax[n % 20, int(n / 20)].set_ylabel('Mean (+/-SD) overall R')
    ax[n % 20, int(n / 20)].set_ylabel('Responding (R)')
    ax[n % 20, int(n / 20)].set_xlabel('Trials')
    ax[n % 20, int(n / 20)].set_title('Sub {0}'.format(sub_ids[n]))   

plt.tight_layout()

plt.savefig(os.path.join('../output/',r'2nd NOS - OS1 Stimulus-Specific, Individual Real and Estimated Responding.svg'), bbox_inches='tight')

In [ ]:
%load_ext watermark
%watermark -v -p conda,jupyterlab,numpy,pandas,theano,pymc3,sklearn